In [155]:
def greedy_best():     
#per ogni endpoint calcolo le differenze tra gli archi verso i datacenter e le cache 
#scelgo le differenze massime
#riempio la cache con video piu richiesti
# ripeto finche tutte le cache non sono piene

    #memoryRemaining[:] = T #istanzio vettore che tiene traccia, per ogni cache, della memoria disponibile
    requestMatrix_c = requestMatrix.copy() #copia della matrice delle richieste
    #la copia è necessaria dato che tale matrice, in questa funzione, verrà modificata 
    assorbimento = np.zeros((C))  
    assorbimento[:] = 1
    num_iteration = 0

    while(num_iteration<C):

        massimo = 0
        i_max = 0
        j_max = 0

        for i in range(E):
            for j in range(C):
                if (latencies[i] - pingMatrix[i][j] > massimo and y[i][j] != 0 and np.any(requestMatrix_c[i]) and assorbimento[j] == 1):     #scelgo la cache rispetto all'endpoint i se esiste un arco tra i due 
                                                                                                                        #ed esistono dei video da usare
                                                                                                                         #e la cache è vuota
                    massimo = latencies[i] - y[i][j]
                    i_max = i #endpoint buono
                    j_max = j #cache buona

        coda_video = np.zeros((2,V))
        
        coda_video[0] = requestMatrix_c[i_max].copy()
        coda_video[1] = size
        coda_video = coda_video.T
        #print(coda_video)
        while(memoryRemaining[j_max] > 0 and coda_video.size != 0 and np.any(coda_video[:,0])):  #knapsack greedy se sono rimasti degli elementi da checkare allora continua

            video_max = np.argmax(coda_video[:,0])  #trovo il video piu richiesto rispetto all endpoint i
            #print(coda_video[:,0])
            size_video = coda_video[video_max][1]

            if(size_video <= memoryRemaining[j_max]):   #se ci entra lo inserisco

                memoryRemaining[j_max] -= size_video    #aggiorno la capacità
                requestMatrix_c[i_max][video_max] = 0  #aggiorno EV, ho soddisfatto la richiesta
                x[video_max][j_max] = 1

             #prova ad inserire il video in una delle cache ancora non piene collegate all'endpoint
                
                

            coda_video = np.delete(coda_video,video_max,axis = 0)    #se il video non entra lo elimino dalla coda

        #cache j_max assorbita
        
        #print(memoryCapacity[j_max])
        assorbimento[j_max] = 0
        num_iteration +=1
        #print(np.any(assorbimento))
        
    checkSolution()  


In [156]:
def localSearch_ScoreC(): #local search con score per le cache, in modo da fare la sceòta che porta più probabilmente ad un miglioramento 
        
        scoreCache = cache_Score() #calcolo lo score per ogni cache. L'indice del vettore rappresenta la cache
        index=-1
        while (index<len(scoreCache)-1): #ciclo esterno che itera finchè non vedo tutte le cache
            index+=1
            check=1                 #variabile che indica se, nelle iterazioni su una cache, sono stati fatti cambiamenti
            cacheIndex = scoreCache[index]  #prelevo l'indice della cache con score più alto
            
            index_video = 0      #indice del video da valutare per l'inserimento o la sostituzione

            while(check):
                dim = 0 #mi serve per controllare se posso effettuare lo scambio
                while(index_video < V and x[index_video][cacheIndex] == 1): #cerco finché non trovo un nuovo video da inserire
                    index_video+=1
                if(index_video>=V): #se ho controllato tutti i video esco
                    break  
                #sezione per aggiungere un video ad una cache se possibile
                if(memoryRemaining[cacheIndex] - size[index_video] >= 0):
                        x[index_video][cacheIndex] = 1    #aggiorno la soluzione
                        if(checkSolution()):              #se mi porta ad un miglioramento, la confermo
                            memoryRemaining[cacheIndex]-=size[index_video]  #aggiorno lo spazio nella cache
                            scoreCache = cache_Score()     #ricalcolo lo score
                            check = 0     #è stata fatta una operazione, quindi esco dal while(check)
                            index=-1
                            
                        else: #se non è una soluzione migliore ripristino la soluzione precedente
                            x[index_video][cacheIndex] = 0
                else:   
                    #sezione per sostituire un video nella cache
                    for i in range(V):                
                        if(x[i][cacheIndex] == 1): #cerco finché non trovo un video presente nella cache
                            dim = memoryRemaining[cacheIndex] + size[i] - size[index_video] #quando lo trovo verifico se è possibile lo scambio
                            if(dim >= 0): #se è possibile aggiorno la soluzione
                                x[index_video][cacheIndex] = 1
                                x[i][cacheIndex] = 0
                                if(checkSolution()): #controllo se è migliore
                                    memoryRemaining[cacheIndex] = dim #se si la confermo
                                    scoreCache = cache_Score()
                                    check = 0
                                    index=-1
                                    break
                                else: #se non è una soluzione migliore ripristino la soluzione precedente
                                    x[index_video][cacheIndex] = 0
                                    x[i][cacheIndex] = 1
                index_video+=1
                if(index_video>=V): #se ho controllato tutti i video esco dal while(1)
                    break 
                

In [157]:
from math import floor
global alpha
def key_get(tupla):
    return tupla[0]

def cache_Score():
    score = [(0,j) for j in range(C)]  #vocabolario con chiave lo score e valore la cache


    for i in range(E):
        for k in range(V):
            for j in range(C):
                if(Matrix[i,j,k] == 1):  #se l'endpoint i fa riferimento alla cache j per il video k
                    score[j] = (score[j][0]+requestMatrix[i][k]*(latencies[i] - pingMatrix[i][j]),j)
                    
#lo score di una cache viene calcolato in base al numero di richieste che soddisfa e che miglioramento porta
#rispetto alla richiesta al data point
                    
    score.sort(key = key_get)  #vengono ordinate le cache in base all'indice
    score2 = [e[1] for e in score]  #prelevo il vettore con lo score per ogni indice
    return score2[:floor(C*alpha)] #restituisco solo la metà del vettore, per valutare solo le cache che mi forniscono un certo miglioramento della soluzione

In [158]:
def checkSolution(): #funzione per verificare la soluzione
    global bestSolution
    global Matrix

    score = 0  #nuovo score
    check=0
    nRequest=0 #richieste totali per calcolare lo score
    delayMatrix=np.zeros((E,V)) #costruisco la matrice dei ritardi da utilizare nel calcolo della soluzione
    savedMatrix=np.zeros((E,V)) #costruisco la matrice del tempo salvato per ogni video


    for i in range(E):
        for k in range(V):
            j_max = -1 
            for j in range(C):
                if(x[k][j]==1 and requestMatrix[i][k] > 0 and delayMatrix[i][k] < pingMatrix[i][j] and y[i][j]==1):
                    #se il video è presente nella cache, c'è una richiesta per quel video e la latenza di quella cache è minore della latenza fin'ora trovata e l'enpoint i fa riferimento alla cache j
                    delayMatrix[i][k] = pingMatrix[i][j] #aggiorno il delay per il video con il ping per la cache
                    savedMatrix[i][k] = latencies[i] - delayMatrix[i][k]  #aggiorno il valore della latenza salvata
                    check = 1  #variabile che serve a capire se c'è un collegamento ad una cache per il nodo i
                    j_max = j  #salvo l'indice della cache ottima
            if(j_max != -1):
                Matrix[i][j_max][k] = 1 #aggiorno il valore in matrix
            if(check == 0 and requestMatrix[i][k] > 0):  #se il nodo i non è legato ad alcuna cache per il video K, ma ci sono richieste per quel video
                delayMatrix[i][k]=latencies[i]   #allora la matrice dei ritardi conterrà il ritardo dal dataCenter
                savedMatrix[i][k]=0      #il ritardo risparmiato è 0
            check=0



    for k in range(E): #per ogni end-point
        for i in range(V): #per ogni video
            score+= requestMatrix[k][i] * savedMatrix[k][i]  
            nRequest += requestMatrix[k][i]
    score=score/nRequest
    score=score*1000

    if(score > bestSolution):
        #print("solBest:",score)
        bestSolution=score
        return 1
    else:
        return 0



In [159]:
def localSearch_Random(): 
            count=0
            limit=200*V  #definisco il limite di iterazioni
            while(count<limit):  
                random.seed()
                cacheIndex = random.randint(0,C-1) #seleziono una cache in modo random 
                video1=random.randint(0,V-1) #seleziono un video random
                video2=random.randint(0,V-1) #seleziono un video da scambiare nella cache
                if(memoryRemaining[cacheIndex]-size[video1]>=0 and x[video1][cacheIndex]==0): #se il video 1 entra nella cache
                    x[video1][cacheIndex]=1 #aggiorno la soluzione
                    if(checkSolution()): 
                        memoryRemaining[cacheIndex]-=size[video1] #se migliore la confermo, altrimenti ripristino
                        count = 0
                    else:
                        x[video1][cacheIndex]=0   
                elif(x[video2][cacheIndex]==1 and memoryRemaining[cacheIndex]+size[video2]-size[video1]>=0 and x[video1][cacheIndex]==0): #se il video 2 è presente nella cache e posso effettuare lo scambio con il video 1
                    #print("entro2")
                    x[video1][cacheIndex]=1
                    x[video2][cacheIndex]=0
                    if(checkSolution()):
                        memoryRemaining[cacheIndex]+=size[video2]
                        memoryRemaining[cacheIndex]-=size[video1]
                        count = 0
                    else:
                        x[video1][cacheIndex]=0
                        x[video2][cacheIndex]=1
                count+=1
                    

In [160]:
def knapsack01(latency_gains,cacheIndex):
#la funzione risolve un problema di Knapsack tramite dinamic programming con approccio bottom up
    W = int(memoryRemaining[cacheIndex])
    n = len(latency_gains)
    video_sizes = size[:floor(V)]

    weights, v = np.hstack(([0], video_sizes)), np.hstack(([0], latency_gains))
    dp = np.zeros((n + 1, W + 1), dtype = np.int64)
    for i in range(1, n + 1):
        for j in range(W + 1):
            if weights[i] > j:
                dp[i, j] = dp[i - 1, j]
            else:
                dp[i, j] = max(dp[i - 1, j - weights[i]] + v[i], dp[i - 1, j])

    i, j, videos, total_weight, total_value = n, W, [], 0, 0
    while i > 0 and j > 0:
        while dp[i, j] == dp[i - 1, j] and i >= 0:
            i -= 1

        if i <= 0:
            break

        videos.append(i - 1)
        j -= weights[i]
        i -= 1

    return videos

def video_Score(cacheIndex):
    score = [(0,j) for j in range(V)]

    for k in range(V):
        if(x[k][cacheIndex] == 0): #se il video non è presente in quella cache
            for i in range(E):
                if(requestMatrix[i][k] != 0 and y[i][cacheIndex] == 1):
                    temp = score[k][0]
                    score[k] = (temp + requestMatrix[i][k] * (latencies[i] - pingMatrix[i][cacheIndex]),k)

    return score[:floor(V)]

def knapsack():
    global x
    scoreCache = cache_Score() #calcolo lo score per ogni cache. L'indice del vettore rappresenta la cache
    n = len(scoreCache)
    index= 0
    while(index<n): #ciclo esterno che itera finchè non vedo tutte le cache
        cacheIndex = scoreCache[index]  #prelevo l'indice della cache con score più alto

        video_gains = video_Score(cacheIndex)

        video_gains2 = [e[0] for e in video_gains]

        video_cache = knapsack01(video_gains2,cacheIndex)


        if(len(video_cache) == 0):
            index += 1
        else:
            for video in video_cache:
                index_video = video_gains[video][1]
                x[index_video][cacheIndex] = 1
                memoryRemaining[cacheIndex] -= size[index_video] 

            scoreCache = cache_Score()
            index = 0
    checkSolution()

In [161]:
import numpy as np
import random 
import time


start = time.time()

bestSolution=0
fileName = "me_at_the_zoo.in"
f = open(fileName)
content = f.readlines() #lista di tutte le linee del file
info = content[0].split() #divido la riga in tutte parole

#salvo ogni parola con il significato che gli è stato attribuito

V = int(info[0]) #numero di video
E = int(info[1]) #numero di end-point
R = int(info[2]) #numero di descrizioni di richieste
C = int(info[3]) #numero di cache server
T = int(info[4]) #capacità di ogni cache server in MB

#siccome la seconda riga contiene le size dei vari video, le salvo

size = [int(a) for a in content[1].split()]

latencies = []  #salvo la latenza di ogni end-point dal dataStore

numCacheServer = [] #salvo il numero di cache server a cui ogni nodo è collegato

pingMatrix = np.zeros((E,C)) #matrice dei ritardi da ogni end-point al cache server

y = np.zeros((E,C))

k=0

for a in range(E):
    row = content[2+k].split()
    latencies.append(int(row[0]))  #salvo la latenza di ogni end-point dal dataStore
    numCacheServer.append(int(row[1])) #salvo il numero di cache a cui è legato ogni end-point
    k+=1
    for i in range(numCacheServer[a]):  #per ogni cache a cui è collegato l'end-point a
        row = content[2+k].split()
        j = int(row[0])    #leggo identificativo cache-center
        pingMatrix[a][j] = int(row[1]) #salvo nella matrice le latenze indicate
        k+=1

    for i in range(C):
        if (pingMatrix[a][i] == 0) :
            pingMatrix[a][i] = 50000000000 #se l'end-point a non è legato alla cache i, introduco nella matrice un valore elevato, in modo da non scegliere quel server
        else :
            y[a][i]=1

requestMatrix = np.zeros((E,V))#matrice delle richieste dell'end-point per ogni video

for a in range(R):
    row = content[2+k].split()
    requestMatrix[int(row[1])][int(row[0])] = int(row[2]) #righe per end-point, colonne per video
    k+=1
    
#finelettura dati input  
#inizio euristica greedy
#partendo dall'endpoint con latenza maggiore, provo a salavre ogni video richiesto nel cache server
memoryRemaining = np.zeros(C) #salvo la memoria usata da ogni cache
memoryRemaining[:] = T
Matrix = np.zeros((E,C,V))  #matrice che indica se l'endpoint E fa riferimento al video V traemite la cache C
alpha = 0.75
x = np.zeros((V,C))  #con Xi,j =1 indico che il video i è presente nella cache j
greedy_best()
print("score Greedy:", bestSolution)
#knapsack()
#print("score knapsack:", bestSolution)
localSearch_ScoreC()
#localSearch_Random()
print("score Finale:", bestSolution)
end = time.time()
print("tempo esecuzione:",end - start)




score Greedy: 294111.7670673247
score Finale: 505306.5624483435
tempo esecuzione: 48.42349100112915


In [47]:
def localSearch_ScoreC_and_V(): 
        scoreCache = cache_Score()
        while(sum(scoreCache) > 0): #ciclo esterno che itera finchè non vedo tutte le cache
            check=1
            cacheIndex = np.argmax(scoreCache)
            scoreCache[cacheIndex] = 0
            
            scoreVideo = video_Score(cacheIndex)

            while(check):
                dim = 0 #mi serve per controllare se posso effettuare lo scambio
                #invece di scorrere tutti i video, inziio a prelevare quelli con lo score più alto per una determinata cache
                if(sum(scoreVideo) > 0): 
                    a = np.argmax(scoreVideo)
                    scoreVideo[a] = 0
                else: #se ho controllato tutti i video esco dal while(1)
                    break 
                #sezione per aggiungere un video ad una cache se possibile
                if(memoryRemaining[cacheIndex] - size[a] >= 0):
                        x[a][cacheIndex] = 1
                        if(checkSolution()):
                            memoryRemaining[cacheIndex]-=size[a]
                            scoreCache = cache_Score()
                            check = 0
                        else: #se non è una soluzione migliore ripristino la soluzione precedente
                            x[a][cacheIndex] = 0
                else:   
                    #sezione per sostituire un video nella cache
                    for i in range(V):               
                        if(x[i][cacheIndex] == 1): #cerco finché non trovo un video presente nella cache
                            dim = memoryRemaining[cacheIndex] + size[i] - size[a]
                            if(dim >= 0):
                                x[a][cacheIndex] = 1
                                x[i][cacheIndex] = 0
                                if(checkSolution()):
                                    memoryRemaining[cacheIndex] = dim
                                    scoreCache = cache_Score()
                                    check = 0
                                    break
                                else: #se non è una soluzione migliore ripristino la soluzione precedente
                                    x[a][cacheIndex] = 0
                                    x[i][cacheIndex] = 1
                                    
                

In [49]:
def greedy_notGood(latencies):
        latency=latencies.copy() #modifico la nuova lista e non quella originale che serve per l'algoritmo, dato che latency la svuoto  
        pingMatrix2=pingMatrix.copy()
        requestMatrix2=requestMatrix.copy()
        video = [] #video a cui fa richiesta un endpoint
        
        lenLatency= len(latency)
        
        while lenLatency >0 :
            b = np.argmax(latency) #prelevo l'endpoint con la latenza maggiore
            latency[b]=0
            lenLatency-=1
        #requestVector=np.zeros(10000)
        #for i in range(E):
        #    requestVector[i]=requestMatrix[i].sum()
        #for i in range(E):
        #    b=np.argmax(requestVector) #prelevo l'endpoint con il maggior numero di richieste
        #    requestVector[b]=0
    
            checkRequest=requestMatrix2[b] #prelevo la riga con le richieste del nodo b per ogni video
            for a in range(V):
                d=np.argmin(checkRequest) #aggiungo alla lista il video con meno richieste
                video.append(d)
                checkRequest[d]=10000000 #modifico così non viene più visto da argmin
            while (len(video)>0):  #per ogni video controllo il cache server a cui posso legarlo
                cacheValutate=0
                k=video.pop()
                cacheRow=pingMatrix2[b] 
                cacheRow=cacheRow[0:C-1].copy()
                cacheMinIndex=np.argmin(cacheRow) #prelevo la cache con il ping minore
                while(y[b][cacheMinIndex] == 0 or (memoryRemaining[cacheMinIndex] - size[k]) < 0): #se l'endpoint non è collegato o il video non entra nella cache
                    cacheRow[cacheMinIndex]=50000000000 
                    cacheMinIndex=np.argmin(cacheRow)
                    cacheValutate+=1
                    if(cacheValutate>C):
                        break
                        
                if(cacheValutate<=C):  
                    memoryRemaining[cacheMinIndex] -= size[k]
                    x[k][cacheMinIndex] = 1
                
                     
        checkSolution()            
        return
                        
        #al termine di questi cicli abbiamo che, a partire dall'endpoint con latenza maggiore, salviamo i video richiesti nelle cache     
        #la prima soluzione sarà sicuramente ammissibile ma molto probabilmente non ottima, formuliamo quindi un euristica di ricerca locale a partire da tale soluzione               
